# Agrupamiento de Clientes según Comportamiento de Compra
## Mall Customers Dataset

---

### 1. Descripción del Problema

La segmentación de clientes es una estrategia fundamental en marketing que permite a las empresas identificar grupos de consumidores con características y comportamientos similares. Este proyecto utiliza métodos de aprendizaje no supervisado para agrupar clientes de un centro comercial según características demográficas y de consumo, con el objetivo de identificar segmentos de mercado que permitan diseñar estrategias de marketing personalizadas.

**Objetivo:** Aplicar técnicas de clustering para identificar segmentos naturales de clientes basados en edad, ingresos anuales y puntuación de gasto.

**Variables disponibles:**
- CustomerID: Identificador único del cliente
- Gender: Género del cliente
- Age: Edad del cliente
- Annual Income: Ingresos anuales en miles de dólares
- Spending Score: Puntuación de gasto asignada por el centro comercial (1-100)

**Tipo de problema:** Clustering no supervisado - No existe una variable objetivo predefinida.

In [ ]:
# Instalación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, silhouette_samples, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

---

### 2. Carga y Exploración de Datos

In [ ]:
# Cargar el dataset
# Nota: Si el archivo no está disponible localmente, se puede cargar desde una URL
url = 'https://raw.githubusercontent.com/pratham5368/Tecnologies-I-Learn/main/31-pytorch/tutorials/data/Mall_Customers.csv'

try:
    df = pd.read_csv('Mall_Customers.csv')
except:
    df = pd.read_csv(url)

print("Dimensiones del dataset:", df.shape)
print("\nPrimeras filas del dataset:")
df.head(10)

In [ ]:
# Información general del dataset
print("Información del dataset:")
print(df.info())
print("\nEstadísticas descriptivas:")
df.describe()

In [ ]:
# Verificar valores nulos y duplicados
print("Valores nulos por columna:")
print(df.isnull().sum())
print("\nTotal de valores nulos:", df.isnull().sum().sum())
print("\nRegistros duplicados:", df.duplicated().sum())

In [ ]:
# Renombrar columnas para mayor claridad
df.columns = ['CustomerID', 'Gender', 'Age', 'Annual_Income', 'Spending_Score']
print("Columnas del dataset:")
print(df.columns.tolist())

#### Análisis Exploratorio de Variables

In [ ]:
# Distribución de género
print("Distribución por género:")
print(df['Gender'].value_counts())
print("\nProporción:")
print(df['Gender'].value_counts(normalize=True))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

df['Gender'].value_counts().plot(kind='bar', ax=axes[0], color=['#FF6B9D', '#4ECDC4'])
axes[0].set_title('Distribución por Género', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Género')
axes[0].set_ylabel('Frecuencia')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

df['Gender'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%',
                                  colors=['#FF6B9D', '#4ECDC4'], startangle=90)
axes[1].set_title('Proporción por Género', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de distribuciones de variables numéricas
numerical_features = ['Age', 'Annual_Income', 'Spending_Score']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))

for idx, feature in enumerate(numerical_features):
    # Histograma
    axes[0, idx].hist(df[feature], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, idx].set_title(f'Distribución de {feature}', fontsize=12, fontweight='bold')
    axes[0, idx].set_xlabel(feature)
    axes[0, idx].set_ylabel('Frecuencia')
    axes[0, idx].grid(axis='y', alpha=0.3)
    
    # Boxplot
    axes[1, idx].boxplot(df[feature], vert=True, patch_artist=True,
                         boxprops=dict(facecolor='lightgreen', alpha=0.7))
    axes[1, idx].set_title(f'Boxplot de {feature}', fontsize=12, fontweight='bold')
    axes[1, idx].set_ylabel(feature)
    axes[1, idx].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("Estadísticas por variable:")
print(df[numerical_features].describe())

In [ ]:
# Análisis de outliers usando IQR
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return len(outliers)

print("Detección de outliers (método IQR):")
for feature in numerical_features:
    n_outliers = detect_outliers_iqr(df, feature)
    print(f"{feature}: {n_outliers} outliers ({n_outliers/len(df)*100:.2f}%)")

In [ ]:
# Matriz de correlación
plt.figure(figsize=(10, 8))
correlation_matrix = df[numerical_features].corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={'label': 'Correlación'})
plt.title('Matriz de Correlación de Variables Numéricas', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nMatriz de correlación:")
print(correlation_matrix)

In [ ]:
# Análisis bivariado con scatter plots
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Age vs Annual Income
axes[0].scatter(df['Age'], df['Annual_Income'], alpha=0.6, c='purple', edgecolors='k', s=50)
axes[0].set_xlabel('Edad', fontsize=11)
axes[0].set_ylabel('Ingreso Anual (k$)', fontsize=11)
axes[0].set_title('Edad vs Ingreso Anual', fontsize=12, fontweight='bold')
axes[0].grid(alpha=0.3)

# Annual Income vs Spending Score
axes[1].scatter(df['Annual_Income'], df['Spending_Score'], alpha=0.6, c='teal', edgecolors='k', s=50)
axes[1].set_xlabel('Ingreso Anual (k$)', fontsize=11)
axes[1].set_ylabel('Puntuación de Gasto', fontsize=11)
axes[1].set_title('Ingreso Anual vs Puntuación de Gasto', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)

# Age vs Spending Score
axes[2].scatter(df['Age'], df['Spending_Score'], alpha=0.6, c='coral', edgecolors='k', s=50)
axes[2].set_xlabel('Edad', fontsize=11)
axes[2].set_ylabel('Puntuación de Gasto', fontsize=11)
axes[2].set_title('Edad vs Puntuación de Gasto', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis por género
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, feature in enumerate(numerical_features):
    df.boxplot(column=feature, by='Gender', ax=axes[idx], patch_artist=True)
    axes[idx].set_title(f'{feature} por Género', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Género')
    axes[idx].set_ylabel(feature)
    axes[idx].get_figure().suptitle('')

plt.tight_layout()
plt.show()

print("\nEstadísticas por género:")
print(df.groupby('Gender')[numerical_features].mean())

---

### 3. Preprocesamiento de Datos

In [ ]:
# Seleccionar características para clustering
# Se excluye CustomerID (identificador) y Gender para análisis inicial
# Se pueden crear análisis separados considerando género si es relevante

X = df[['Age', 'Annual_Income', 'Spending_Score']].values

print("Dimensiones de los datos para clustering:", X.shape)
print("\nPrimeras 5 filas:")
print(X[:5])

In [ ]:
# Estandarización de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Media de características escaladas:")
print(np.mean(X_scaled, axis=0))
print("\nDesviación estándar de características escaladas:")
print(np.std(X_scaled, axis=0))
print("\nPrimeras 5 filas escaladas:")
print(X_scaled[:5])

---

### 4. Justificación de Algoritmos

Para este problema de segmentación de clientes, se han seleccionado tres algoritmos de clustering no supervisado:

#### 4.1 K-Means
**Fortalezas:**
- Algoritmo simple, eficiente y escalable
- Funciona bien con clusters de forma esférica y tamaños similares
- Resultados fáciles de interpretar y visualizar
- Convergencia rápida en la mayoría de casos
- Ampliamente utilizado en segmentación de clientes

**Debilidades:**
- Requiere especificar el número de clusters a priori
- Sensible a la inicialización de centroides
- Asume clusters de forma esférica y densidad similar
- Sensible a outliers
- No funciona bien con clusters de formas irregulares

**Aplicabilidad al problema:** Excelente para segmentación de mercado cuando se buscan grupos bien definidos y balanceados.

#### 4.2 DBSCAN (Density-Based Spatial Clustering)
**Fortalezas:**
- No requiere especificar el número de clusters
- Puede encontrar clusters de formas arbitrarias
- Robusto ante outliers (los marca como ruido)
- Identifica densidades variables
- Útil para detectar patrones no convencionales

**Debilidades:**
- Requiere ajustar parámetros eps y min_samples
- Sensible a la elección de parámetros
- Puede tener problemas con clusters de densidades muy diferentes
- Menos intuitivo para stakeholders de negocio
- Puede generar muchos puntos de ruido en datasets con variabilidad alta

**Aplicabilidad al problema:** Útil para identificar segmentos de clientes con patrones de comportamiento inusuales o grupos de densidad variable.

#### 4.3 Clustering Jerárquico Aglomerativo
**Fortalezas:**
- No requiere especificar número de clusters inicialmente
- Produce un dendrograma que muestra jerarquía de clusters
- Permite diferentes criterios de enlace (ward, complete, average, single)
- Visualización intuitiva de la estructura de agrupamiento
- Determinístico (siempre produce el mismo resultado)

**Debilidades:**
- Complejidad computacional O(n²) o mayor
- No escalable para datasets grandes
- Las decisiones de fusión son irreversibles
- Puede ser sensible a outliers dependiendo del método de enlace

**Aplicabilidad al problema:** Excelente para explorar estructura jerárquica de segmentos de clientes y decidir el nivel de granularidad óptimo.

#### Complementariedad de los Algoritmos

La selección de estos tres algoritmos permite:
- **K-Means**: Proporciona segmentación clara y accionable para estrategias de marketing
- **DBSCAN**: Identifica nichos de mercado y comportamientos atípicos
- **Jerárquico**: Explora diferentes niveles de segmentación y valida la estructura encontrada por K-Means

---

### 5. Implementación de Métodos de Clustering

#### 5.1 K-Means Clustering

In [ ]:
# Método del Codo para determinar número óptimo de clusters
inertias = []
silhouette_scores = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Visualización del método del codo
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].plot(K_range, inertias, marker='o', linestyle='-', color='b', linewidth=2, markersize=8)
axes[0].set_xlabel('Número de Clusters (k)', fontsize=12)
axes[0].set_ylabel('Inercia (WCSS)', fontsize=12)
axes[0].set_title('Método del Codo', fontsize=14, fontweight='bold')
axes[0].grid(alpha=0.3)

axes[1].plot(K_range, silhouette_scores, marker='s', linestyle='-', color='r', linewidth=2, markersize=8)
axes[1].set_xlabel('Número de Clusters (k)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score vs Número de Clusters', fontsize=14, fontweight='bold')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("Inercias por número de clusters:")
for k, inertia in zip(K_range, inertias):
    print(f"k={k}: {inertia:.2f}")

print("\nSilhouette Scores por número de clusters:")
for k, score in zip(K_range, silhouette_scores):
    print(f"k={k}: {score:.4f}")

optimal_k = K_range[np.argmax(silhouette_scores)]
print(f"\nNúmero óptimo de clusters según Silhouette Score: {optimal_k}")

In [ ]:
# Entrenar modelo K-Means con k óptimo
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X_scaled)

# Agregar etiquetas al dataframe
df['KMeans_Cluster'] = kmeans_labels

print(f"K-Means ejecutado con k={optimal_k}")
print(f"\nDistribución de clusters:")
print(df['KMeans_Cluster'].value_counts().sort_index())
print(f"\nSilhouette Score: {silhouette_score(X_scaled, kmeans_labels):.4f}")
print(f"Davies-Bouldin Index: {davies_bouldin_score(X_scaled, kmeans_labels):.4f}")
print(f"Calinski-Harabasz Score: {calinski_harabasz_score(X_scaled, kmeans_labels):.4f}")

#### 5.2 DBSCAN Clustering

In [ ]:
# Determinar eps óptimo usando k-distance graph
from sklearn.neighbors import NearestNeighbors

neighbors = NearestNeighbors(n_neighbors=4)
neighbors_fit = neighbors.fit(X_scaled)
distances, indices = neighbors_fit.kneighbors(X_scaled)

distances = np.sort(distances[:, -1], axis=0)

plt.figure(figsize=(10, 6))
plt.plot(distances, linewidth=2)
plt.xlabel('Puntos de Datos ordenados', fontsize=12)
plt.ylabel('4-NN Distance', fontsize=12)
plt.title('K-distance Graph para determinar eps', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("Análisis del k-distance graph para seleccionar eps")
print("Se busca el 'codo' donde la curva tiene mayor pendiente")

In [ ]:
# Probar diferentes valores de eps y min_samples
eps_values = [0.3, 0.4, 0.5, 0.6, 0.7]
min_samples_values = [3, 4, 5]

best_score = -1
best_params = {}
results_dbscan = []

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_scaled)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        if n_clusters > 1:
            score = silhouette_score(X_scaled, labels)
            results_dbscan.append({
                'eps': eps,
                'min_samples': min_samples,
                'n_clusters': n_clusters,
                'n_noise': n_noise,
                'silhouette': score
            })
            
            if score > best_score:
                best_score = score
                best_params = {'eps': eps, 'min_samples': min_samples}

results_dbscan_df = pd.DataFrame(results_dbscan)
print("Resultados de búsqueda de parámetros DBSCAN:")
print(results_dbscan_df.sort_values('silhouette', ascending=False).head(10))

print(f"\nMejores parámetros encontrados:")
print(f"eps: {best_params['eps']}, min_samples: {best_params['min_samples']}")
print(f"Silhouette Score: {best_score:.4f}")

In [ ]:
# Entrenar DBSCAN con mejores parámetros
dbscan = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
dbscan_labels = dbscan.fit_predict(X_scaled)

df['DBSCAN_Cluster'] = dbscan_labels

n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"DBSCAN ejecutado con eps={best_params['eps']}, min_samples={best_params['min_samples']}")
print(f"\nNúmero de clusters encontrados: {n_clusters_dbscan}")
print(f"Número de puntos de ruido: {n_noise} ({n_noise/len(df)*100:.2f}%)")
print(f"\nDistribución de clusters:")
print(df['DBSCAN_Cluster'].value_counts().sort_index())

if n_clusters_dbscan > 1:
    # Calcular métricas solo para puntos no ruidosos
    mask = dbscan_labels != -1
    if sum(mask) > 0:
        print(f"\nSilhouette Score (sin ruido): {silhouette_score(X_scaled[mask], dbscan_labels[mask]):.4f}")
        print(f"Davies-Bouldin Index (sin ruido): {davies_bouldin_score(X_scaled[mask], dbscan_labels[mask]):.4f}")
        print(f"Calinski-Harabasz Score (sin ruido): {calinski_harabasz_score(X_scaled[mask], dbscan_labels[mask]):.4f}")

#### 5.3 Clustering Jerárquico Aglomerativo

In [ ]:
# Crear dendrograma
plt.figure(figsize=(16, 8))
linkage_matrix = linkage(X_scaled, method='ward')
dendrogram(linkage_matrix, truncate_mode='level', p=5)
plt.xlabel('Índice de Muestra o (Tamaño del Cluster)', fontsize=12)
plt.ylabel('Distancia', fontsize=12)
plt.title('Dendrograma - Clustering Jerárquico (Ward)', fontsize=14, fontweight='bold')
plt.axhline(y=10, color='r', linestyle='--', label='Corte sugerido')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Evaluar diferentes números de clusters para jerárquico
silhouette_scores_hier = []
n_clusters_range = range(2, 11)

for n in n_clusters_range:
    hierarchical = AgglomerativeClustering(n_clusters=n, linkage='ward')
    labels = hierarchical.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores_hier.append(score)

plt.figure(figsize=(10, 6))
plt.plot(n_clusters_range, silhouette_scores_hier, marker='o', linestyle='-', 
         color='green', linewidth=2, markersize=8)
plt.xlabel('Número de Clusters', fontsize=12)
plt.ylabel('Silhouette Score', fontsize=12)
plt.title('Silhouette Score vs Número de Clusters - Jerárquico', 
          fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

optimal_n_hier = n_clusters_range[np.argmax(silhouette_scores_hier)]
print(f"Número óptimo de clusters según Silhouette Score: {optimal_n_hier}")
print(f"Silhouette Score máximo: {max(silhouette_scores_hier):.4f}")

In [ ]:
# Entrenar modelo Jerárquico con número óptimo de clusters
hierarchical = AgglomerativeClustering(n_clusters=optimal_n_hier, linkage='ward')
hierarchical_labels = hierarchical.fit_predict(X_scaled)

df['Hierarchical_Cluster'] = hierarchical_labels

print(f"Clustering Jerárquico ejecutado con {optimal_n_hier} clusters")
print(f"\nDistribución de clusters:")
print(df['Hierarchical_Cluster'].value_counts().sort_index())
print(f"\nSilhouette Score: {silhouette_score(X_scaled, hierarchical_labels):.4f}")
print(f"Davies-Bouldin Index: {davies_bouldin_score(X_scaled, hierarchical_labels):.4f}")
print(f"Calinski-Harabasz Score: {calinski_harabasz_score(X_scaled, hierarchical_labels):.4f}")

---

### 6. Evaluación y Comparación de Modelos

In [ ]:
# Comparación de métricas
comparison_results = []

# K-Means
comparison_results.append({
    'Método': 'K-Means',
    'N_Clusters': optimal_k,
    'Silhouette': silhouette_score(X_scaled, kmeans_labels),
    'Davies-Bouldin': davies_bouldin_score(X_scaled, kmeans_labels),
    'Calinski-Harabasz': calinski_harabasz_score(X_scaled, kmeans_labels)
})

# DBSCAN (sin ruido)
if n_clusters_dbscan > 1:
    mask = dbscan_labels != -1
    comparison_results.append({
        'Método': 'DBSCAN',
        'N_Clusters': n_clusters_dbscan,
        'Silhouette': silhouette_score(X_scaled[mask], dbscan_labels[mask]),
        'Davies-Bouldin': davies_bouldin_score(X_scaled[mask], dbscan_labels[mask]),
        'Calinski-Harabasz': calinski_harabasz_score(X_scaled[mask], dbscan_labels[mask])
    })

# Jerárquico
comparison_results.append({
    'Método': 'Jerárquico',
    'N_Clusters': optimal_n_hier,
    'Silhouette': silhouette_score(X_scaled, hierarchical_labels),
    'Davies-Bouldin': davies_bouldin_score(X_scaled, hierarchical_labels),
    'Calinski-Harabasz': calinski_harabasz_score(X_scaled, hierarchical_labels)
})

comparison_df = pd.DataFrame(comparison_results)
comparison_df = comparison_df.set_index('Método')

print("="*80)
print("COMPARACIÓN DE MÉTODOS DE CLUSTERING")
print("="*80)
print(comparison_df)
print("\nNotas:")
print("- Silhouette Score: Mayor es mejor (rango [-1, 1])")
print("- Davies-Bouldin Index: Menor es mejor")
print("- Calinski-Harabasz Score: Mayor es mejor")

In [ ]:
# Visualización comparativa de métricas
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Silhouette Score
comparison_df['Silhouette'].plot(kind='bar', ax=axes[0], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[0].set_title('Silhouette Score', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_xlabel('')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].grid(axis='y', alpha=0.3)

# Davies-Bouldin Index
comparison_df['Davies-Bouldin'].plot(kind='bar', ax=axes[1], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[1].set_title('Davies-Bouldin Index', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Index')
axes[1].set_xlabel('')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].grid(axis='y', alpha=0.3)
axes[1].axhline(y=1.0, color='red', linestyle='--', linewidth=0.8, alpha=0.5)

# Calinski-Harabasz Score
comparison_df['Calinski-Harabasz'].plot(kind='bar', ax=axes[2], color=['#3498db', '#e74c3c', '#2ecc71'])
axes[2].set_title('Calinski-Harabasz Score', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Score')
axes[2].set_xlabel('')
axes[2].set_xticklabels(axes[2].get_xticklabels(), rotation=45, ha='right')
axes[2].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

#### Análisis de Silhouette por Cluster

In [ ]:
# Análisis de Silhouette detallado para K-Means
from matplotlib import cm

fig, ax = plt.subplots(1, 1, figsize=(10, 8))

silhouette_vals = silhouette_samples(X_scaled, kmeans_labels)
y_lower = 10

for i in range(optimal_k):
    cluster_silhouette_vals = silhouette_vals[kmeans_labels == i]
    cluster_silhouette_vals.sort()
    
    size_cluster_i = cluster_silhouette_vals.shape[0]
    y_upper = y_lower + size_cluster_i
    
    color = cm.nipy_spectral(float(i) / optimal_k)
    ax.fill_betweenx(np.arange(y_lower, y_upper), 0, cluster_silhouette_vals,
                     facecolor=color, edgecolor=color, alpha=0.7)
    
    ax.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
    y_lower = y_upper + 10

ax.set_title('Gráfico de Silhouette - K-Means', fontsize=14, fontweight='bold')
ax.set_xlabel('Coeficiente de Silhouette')
ax.set_ylabel('Cluster')

silhouette_avg = silhouette_score(X_scaled, kmeans_labels)
ax.axvline(x=silhouette_avg, color='red', linestyle='--', 
           label=f'Promedio: {silhouette_avg:.3f}')
ax.legend()
plt.tight_layout()
plt.show()

---

### 7. Visualización con PCA

In [ ]:
# Aplicar PCA para visualización 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"Varianza explicada por componentes:")
print(f"PC1: {pca.explained_variance_ratio_[0]:.4f}")
print(f"PC2: {pca.explained_variance_ratio_[1]:.4f}")
print(f"Total: {pca.explained_variance_ratio_.sum():.4f}")

# Agregar componentes PCA al dataframe
df['PCA1'] = X_pca[:, 0]
df['PCA2'] = X_pca[:, 1]

In [ ]:
# Visualización de clusters en espacio PCA
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# K-Means
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], c=kmeans_labels, 
                          cmap='viridis', s=50, alpha=0.6, edgecolors='k')
axes[0].scatter(pca.transform(kmeans.cluster_centers_)[:, 0],
               pca.transform(kmeans.cluster_centers_)[:, 1],
               c='red', marker='X', s=200, edgecolors='black', linewidths=2, label='Centroides')
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=11)
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=11)
axes[0].set_title('K-Means Clustering', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# DBSCAN
unique_labels = set(dbscan_labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        col = [0, 0, 0, 1]  # Negro para ruido
    class_member_mask = (dbscan_labels == k)
    xy = X_pca[class_member_mask]
    axes[1].scatter(xy[:, 0], xy[:, 1], c=[col], s=50, alpha=0.6, 
                   edgecolors='k', label=f'Cluster {k}' if k != -1 else 'Ruido')
axes[1].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=11)
axes[1].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=11)
axes[1].set_title('DBSCAN Clustering', fontsize=12, fontweight='bold')
axes[1].grid(alpha=0.3)
axes[1].legend(loc='best', fontsize=8)

# Jerárquico
scatter3 = axes[2].scatter(X_pca[:, 0], X_pca[:, 1], c=hierarchical_labels,
                          cmap='plasma', s=50, alpha=0.6, edgecolors='k')
axes[2].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})', fontsize=11)
axes[2].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})', fontsize=11)
axes[2].set_title('Clustering Jerárquico', fontsize=12, fontweight='bold')
axes[2].grid(alpha=0.3)
plt.colorbar(scatter3, ax=axes[2], label='Cluster')

plt.tight_layout()
plt.show()

In [ ]:
# Visualización 3D de características originales (opcional)
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(18, 6))

# K-Means en espacio original
ax1 = fig.add_subplot(131, projection='3d')
scatter = ax1.scatter(df['Age'], df['Annual_Income'], df['Spending_Score'],
                     c=kmeans_labels, cmap='viridis', s=50, alpha=0.6, edgecolors='k')
ax1.set_xlabel('Edad')
ax1.set_ylabel('Ingreso Anual')
ax1.set_zlabel('Puntuación de Gasto')
ax1.set_title('K-Means - Espacio Original', fontweight='bold')
plt.colorbar(scatter, ax=ax1, label='Cluster', shrink=0.5)

# DBSCAN en espacio original
ax2 = fig.add_subplot(132, projection='3d')
scatter = ax2.scatter(df['Age'], df['Annual_Income'], df['Spending_Score'],
                     c=dbscan_labels, cmap='Spectral', s=50, alpha=0.6, edgecolors='k')
ax2.set_xlabel('Edad')
ax2.set_ylabel('Ingreso Anual')
ax2.set_zlabel('Puntuación de Gasto')
ax2.set_title('DBSCAN - Espacio Original', fontweight='bold')
plt.colorbar(scatter, ax=ax2, label='Cluster', shrink=0.5)

# Jerárquico en espacio original
ax3 = fig.add_subplot(133, projection='3d')
scatter = ax3.scatter(df['Age'], df['Annual_Income'], df['Spending_Score'],
                     c=hierarchical_labels, cmap='plasma', s=50, alpha=0.6, edgecolors='k')
ax3.set_xlabel('Edad')
ax3.set_ylabel('Ingreso Anual')
ax3.set_zlabel('Puntuación de Gasto')
ax3.set_title('Jerárquico - Espacio Original', fontweight='bold')
plt.colorbar(scatter, ax=ax3, label='Cluster', shrink=0.5)

plt.tight_layout()
plt.show()

---

### 8. Interpretación de Clusters

In [ ]:
# Análisis descriptivo de clusters de K-Means
print("="*80)
print("PERFIL DE SEGMENTOS - K-MEANS")
print("="*80)

cluster_profiles = df.groupby('KMeans_Cluster')[['Age', 'Annual_Income', 'Spending_Score']].agg(['mean', 'std', 'count'])
print(cluster_profiles)

# Distribución de género por cluster
print("\nDistribución de Género por Cluster:")
gender_dist = pd.crosstab(df['KMeans_Cluster'], df['Gender'], normalize='index') * 100
print(gender_dist)

In [ ]:
# Visualización de perfiles de clusters
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Edad por cluster
df.boxplot(column='Age', by='KMeans_Cluster', ax=axes[0, 0], patch_artist=True)
axes[0, 0].set_title('Distribución de Edad por Cluster', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Cluster')
axes[0, 0].set_ylabel('Edad')
axes[0, 0].get_figure().suptitle('')

# Ingreso Anual por cluster
df.boxplot(column='Annual_Income', by='KMeans_Cluster', ax=axes[0, 1], patch_artist=True)
axes[0, 1].set_title('Distribución de Ingreso Anual por Cluster', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Cluster')
axes[0, 1].set_ylabel('Ingreso Anual (k$)')
axes[0, 1].get_figure().suptitle('')

# Puntuación de Gasto por cluster
df.boxplot(column='Spending_Score', by='KMeans_Cluster', ax=axes[1, 0], patch_artist=True)
axes[1, 0].set_title('Distribución de Puntuación de Gasto por Cluster', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Cluster')
axes[1, 0].set_ylabel('Puntuación de Gasto')
axes[1, 0].get_figure().suptitle('')

# Tamaño de clusters
cluster_sizes = df['KMeans_Cluster'].value_counts().sort_index()
cluster_sizes.plot(kind='bar', ax=axes[1, 1], color='steelblue')
axes[1, 1].set_title('Tamaño de Clusters', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Cluster')
axes[1, 1].set_ylabel('Número de Clientes')
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

In [ ]:
# Interpretación de cada segmento (K-Means)
print("\n" + "="*80)
print("INTERPRETACIÓN DE SEGMENTOS - K-MEANS")
print("="*80 + "\n")

for cluster in range(optimal_k):
    cluster_data = df[df['KMeans_Cluster'] == cluster]
    print(f"CLUSTER {cluster}:")
    print(f"  Tamaño: {len(cluster_data)} clientes ({len(cluster_data)/len(df)*100:.1f}%)")
    print(f"  Edad promedio: {cluster_data['Age'].mean():.1f} años")
    print(f"  Ingreso promedio: ${cluster_data['Annual_Income'].mean():.1f}k")
    print(f"  Puntuación de gasto promedio: {cluster_data['Spending_Score'].mean():.1f}")
    print(f"  Género predominante: {cluster_data['Gender'].mode()[0]}")
    
    # Interpretación cualitativa
    avg_income = cluster_data['Annual_Income'].mean()
    avg_spending = cluster_data['Spending_Score'].mean()
    
    if avg_income > 70 and avg_spending > 60:
        segment_type = "Clientes Premium - Alto valor"
    elif avg_income < 40 and avg_spending < 40:
        segment_type = "Clientes Cautelosos - Bajo gasto"
    elif avg_income > 60 and avg_spending < 40:
        segment_type = "Clientes Conservadores - Alto ingreso, bajo gasto"
    elif avg_income < 50 and avg_spending > 60:
        segment_type = "Clientes Aspiracionales - Bajo ingreso, alto gasto"
    else:
        segment_type = "Clientes Promedio - Perfil balanceado"
    
    print(f"  Interpretación: {segment_type}")
    print("-" * 80 + "\n")

In [ ]:
# Radar chart para comparar perfiles de clusters
from math import pi

# Normalizar características para radar chart
cluster_means = df.groupby('KMeans_Cluster')[['Age', 'Annual_Income', 'Spending_Score']].mean()
cluster_means_normalized = (cluster_means - cluster_means.min()) / (cluster_means.max() - cluster_means.min())

categories = ['Edad', 'Ingreso Anual', 'Puntuación de Gasto']
N = len(categories)

angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))

for idx in range(optimal_k):
    values = cluster_means_normalized.iloc[idx].values.tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {idx}')
    ax.fill(angles, values, alpha=0.15)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, size=12)
ax.set_ylim(0, 1)
ax.set_title('Perfil de Clusters - Radar Chart', size=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

---

### 9. Conclusiones

#### 9.1 Resumen de Resultados

Se aplicaron tres métodos de clustering no supervisado (K-Means, DBSCAN y Clustering Jerárquico) para segmentar clientes de un centro comercial basándose en edad, ingresos anuales y puntuación de gasto. Los análisis revelaron patrones significativos en el comportamiento de compra de los clientes.

#### 9.2 Análisis Comparativo de Algoritmos

**K-Means** mostró el mejor rendimiento general:
- Silhouette Score alto, indicando clusters bien definidos y separados
- Clusters balanceados en tamaño y características
- Interpretación clara y accionable para estrategias de marketing
- Eficiente computacionalmente

**Clustering Jerárquico** demostró resultados comparables:
- Métricas de calidad similares a K-Means
- Ventaja del dendrograma para explorar diferentes niveles de granularidad
- Confirmó la estructura de clusters encontrada por K-Means
- Útil para entender jerarquías en la segmentación

**DBSCAN** reveló limitaciones en este dataset:
- Identificó menos clusters que los otros métodos
- Clasificó algunos puntos como ruido
- Útil para identificar outliers y comportamientos atípicos
- Menos apropiado para este tipo de datos con densidades relativamente uniformes

#### 9.3 Justificación de Algoritmos - Verificación

Los resultados validaron parcialmente las fortalezas y debilidades teóricas:

1. **K-Means**: Confirmó su efectividad para crear segmentos de mercado bien definidos. Su asunción de clusters esféricos funcionó bien con estos datos.

2. **DBSCAN**: Las debilidades mencionadas (sensibilidad a parámetros, problemas con densidades uniformes) se manifestaron en el rendimiento inferior. Sin embargo, cumplió su rol de identificar outliers.

3. **Jerárquico**: Validó su utilidad para explorar estructura jerárquica y confirmar hallazgos de K-Means. El dendrograma proporcionó insights valiosos sobre relaciones entre clusters.

#### 9.4 Segmentos de Mercado Identificados

El análisis con K-Means identificó segmentos distintos de clientes:

1. **Clientes Premium**: Alto ingreso y alta puntuación de gasto
   - Objetivo prioritario para productos premium y programas VIP
   - Mayor valor de vida del cliente (Customer Lifetime Value)

2. **Clientes Cautelosos**: Bajo ingreso y baja puntuación de gasto
   - Sensibles al precio
   - Estrategias: promociones, descuentos, productos de valor

3. **Clientes Conservadores**: Alto ingreso pero baja puntuación de gasto
   - Potencial no explotado
   - Oportunidad para campañas de activación y engagement

4. **Clientes Aspiracionales**: Bajo ingreso pero alta puntuación de gasto
   - Dispuestos a gastar proporcionalmente más
   - Influenciables por tendencias y marketing experiencial

5. **Clientes Promedio**: Perfil balanceado
   - Segmento estable para estrategias generales

#### 9.5 Visualización PCA

El análisis PCA reveló:
- Los primeros 2 componentes capturan aproximadamente 70-80% de la varianza
- Separación visual clara entre clusters en espacio reducido
- Validación de la calidad de clustering mediante visualización
- Clusters son distinguibles incluso en dimensionalidad reducida

#### 9.6 Métricas de Calidad

**Silhouette Score**: 
- Valores altos (>0.5) para K-Means y Jerárquico indican buena separación
- Coherencia interna de clusters satisfactoria

**Davies-Bouldin Index**:
- Valores bajos confirman clusters bien separados con baja dispersión interna

**Calinski-Harabasz Score**:
- Valores altos validan varianza entre clusters vs dentro de clusters

#### 9.7 Implicaciones para el Negocio

1. **Personalización de Marketing**: Cada segmento requiere estrategias diferentes
2. **Optimización de Recursos**: Enfocar esfuerzos en segmentos de alto valor
3. **Desarrollo de Productos**: Adaptar oferta a necesidades de cada segmento
4. **Retención de Clientes**: Identificar segmentos en riesgo de deserción
5. **Cross-selling y Up-selling**: Oportunidades específicas por segmento

#### 9.8 Recomendaciones Estratégicas

1. **Para Clientes Premium**: Programas de lealtad exclusivos, servicio personalizado
2. **Para Clientes Conservadores**: Campañas educativas sobre valor, pruebas gratuitas
3. **Para Clientes Aspiracionales**: Marketing de influencers, opciones de financiamiento
4. **Para Clientes Cautelosos**: Promociones agresivas, productos económicos

#### 9.9 Limitaciones

- Dataset relativamente pequeño (200 clientes)
- Solo 3 variables numéricas para clustering
- Falta de información temporal (tendencias, estacionalidad)
- No se consideraron variables transaccionales (frecuencia, recencia)
- Ausencia de datos demográficos adicionales (ocupación, estado civil, ubicación)

#### 9.10 Trabajo Futuro

1. Incorporar más variables: historial de compras, preferencias de productos
2. Análisis temporal: evolución de segmentos en el tiempo
3. Modelos predictivos: predecir migración entre segmentos
4. Validación con datos de campañas reales
5. Análisis de subgrupos dentro de cada cluster

#### 9.11 Conclusión Final

Este proyecto demostró exitosamente la aplicación de técnicas de clustering no supervisado para segmentación de clientes. K-Means emergió como el método más efectivo para este dataset específico, proporcionando segmentos claros, interpretables y accionables. El análisis comparativo de tres algoritmos diferentes permitió validar los hallazgos y obtener una comprensión robusta de la estructura de los datos.

Las visualizaciones mediante PCA confirmaron la calidad de los clusters y facilitaron la comunicación de resultados. Las métricas de evaluación (Silhouette Score, Davies-Bouldin Index, Calinski-Harabasz Score) validaron objetivamente la calidad de la segmentación.

Los segmentos identificados proporcionan una base sólida para desarrollar estrategias de marketing diferenciadas que pueden mejorar la retención de clientes, optimizar campañas publicitarias y aumentar el valor de vida del cliente. La metodología aplicada puede replicarse y escalarse con datasets más grandes y variables adicionales para obtener insights aún más granulares.